# Generate FIR Filter Coefficients

Demonstrates how to generate precomputed/predefined FIR filter coefficients for various resampling scenarios based on input and output sample rates. The coefficients are formatted in a convenient way to facilitate adding them to a C/C++ file as compile-time constants. 

In [22]:
from fractions import Fraction
import numpy as np
from scipy import signal

INPUT_SAMPLE_RATE = 2000000
OUTPUT_SAMPLE_RATE = 200e3

resampling_ratio = OUTPUT_SAMPLE_RATE / INPUT_SAMPLE_RATE
ratio = Fraction("%.12f" % (resampling_ratio)).limit_denominator()
p = ratio.numerator
q = ratio.denominator
pqmax = max(p, q)

# cutoff frequency of the lowpass filter at the high (upsampled) rate
cutoff_freq = 1 / 2 / pqmax
filter_order = 2 * 10 * pqmax + 1
filter_delay = int((filter_order - 1) / p / 2)

filter_coeffs = float(p) * signal.firls(filter_order, [0, 2.0 * cutoff_freq, 2.0 * cutoff_freq, 1.0], [1.0, 1.0, 0.0, 0.0])
filter_coeffs = filter_coeffs * signal.kaiser(filter_order, beta=5)

next_line = "             "
for coeff_idx in range(len(filter_coeffs)):
    next_line += "{%+01.08f, %01.08f}" % (filter_coeffs[coeff_idx].real, filter_coeffs[coeff_idx].imag)
    if coeff_idx % 4 == 3:
        print(next_line + ","); next_line = "             "
    else:
        next_line += ", "

print(next_line)

             {-0.00000000, 0.00000000}, {-0.00004115, 0.00000000}, {-0.00008854, 0.00000000}, {-0.00013692, 0.00000000},
             {-0.00017983, 0.00000000}, {-0.00021018, 0.00000000}, {-0.00022121, 0.00000000}, {-0.00020740, 0.00000000},
             {-0.00016548, 0.00000000}, {-0.00009523, 0.00000000}, {+0.00000000, 0.00000000}, {+0.00011310, 0.00000000},
             {+0.00023354, 0.00000000}, {+0.00034813, 0.00000000}, {+0.00044227, 0.00000000}, {+0.00050153, 0.00000000},
             {+0.00051346, 0.00000000}, {+0.00046933, 0.00000000}, {+0.00036580, 0.00000000}, {+0.00020599, 0.00000000},
             {-0.00000000, 0.00000000}, {-0.00023529, 0.00000000}, {-0.00047738, 0.00000000}, {-0.00069996, 0.00000000},
             {-0.00087554, 0.00000000}, {-0.00097846, 0.00000000}, {-0.00098800, 0.00000000}, {-0.00089141, 0.00000000},
             {-0.00068626, 0.00000000}, {-0.00038194, 0.00000000}, {-0.00000000, 0.00000000}, {+0.00042694, 0.00000000},
             {+0.00085756, 0.000